# Exp 23: Multi-Signal Ranking & Query-Time Enhancement

## Motivation

Exp 22 showed that static-fact `values_early_layers` (VEL) provides **zero ranking benefit**
(AUC 0.832 vs bare 0.841 with PMI). The hypothesis: a static, query-independent prefix cannot
create query-specific relevance discrimination. This experiment tests 4 new ideas:

1. **Cached Answer Templates** — Score a relevance template instead of the actual answer
2. **Query-Time Value Injection (QVI)** — Inject query info at scoring time via value blending
3. **Two-Stage Pipeline** — Bare PMI rank → re-rank top-k with oracle VEL
4. **Query-Clustered Priming** — 5 intent-specialized prefixes, route to best

## Design

| Parameter | Value |
|-----------|-------|
| Model | Gemma 3 4B (`google/gemma-3-4b-it`, 4-bit, bfloat16) |
| Dataset | MS MARCO v1.1 validation (same filtering as Exp 22) |
| Scale | 200 queries × ~8.5 passages ≈ 1,700 passages |
| Checkpoint | Every 5 queries |

### 13 Cache Conditions

| # | Name | Cache Build | FP Cost |
|---|------|-------------|--------|
| 1 | `bare` | `[BOS][doc]` | 1 FP (shared) |
| 2 | `fact_vel` | VEL with static_factual, L0-16 | 1 FP |
| 3 | `def_vel` | VEL with static_definitional, L0-16 | 1 FP |
| 4 | `proc_vel` | VEL with static_procedural, L0-16 | 1 FP |
| 5 | `quant_vel` | VEL with static_quantitative, L0-16 | 1 FP |
| 6 | `prob_vel` | VEL with static_problem, L0-16 | 1 FP |
| 7 | `oracle_vel` | VEL with actual query, L0-16 | 1 FP |
| 8 | `oracle_vel_low` | VEL with actual query, L0-8 | reuse #7 FP |
| 9 | `oracle_interp` | `interpolate_values(bare, oracle_corr, α=0.25)` all layers | reuse #7 FP |
| 10 | `oracle_full` | Full `[BOS][query][doc]` context (no truncation) | reuse #7 FP |
| 11 | `qvi_010` | QVI α=0.10, L0-16 | reuse bare+query |
| 12 | `qvi_025` | QVI α=0.25, L0-16 | reuse bare+query |
| 13 | `qvi_050` | QVI α=0.50, L0-16 | reuse bare+query |

### 3 Scoring Targets

| Target | query_prompt | answer_text | Purpose |
|--------|-------------|-------------|--------|
| `answer` | `"\nQuery: {query}\nAnswer:"` | `" {answer}"` | Standard: can cache predict the answer? |
| `qdoc` | `"\nThis document is about:"` | `" {query}"` | Bidirectional: does doc predict the query? |
| `relevance` | `"\nQuery: {query}\nIs this document relevant?"` | `" Yes, this document is relevant to the query"` | Template: explicit relevance judgment |

### Per-Passage: 13 conditions × 3 targets = 39 scores

### Analysis-Time Derived Scores (no extra data)

1. **PMI versions**: `score - baseline` for all 39 combinations
2. **Best-intent routing**: `min(fact, def, proc, quant, prob)` per passage per target
3. **Two-stage pipeline**: Rank by bare PMI → re-rank top-k with oracle_vel PMI
4. **Per-query intent routing**: For each query, pick intent with lowest mean NLL

In [1]:
# Cell 1: Setup & Imports
import os
os.umask(0o000)
os.environ['HF_TOKEN'] = os.environ.get('HF_TOKEN', '')

import sys
import json
import time
import numpy as np
import torch
import gc
from pathlib import Path

SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)

RESULTS_DIR = Path("results/exp23")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
(RESULTS_DIR / "figures").mkdir(parents=True, exist_ok=True)

CHECKPOINT_PATH = RESULTS_DIR / "checkpoint.json"
FINAL_RESULTS_PATH = RESULTS_DIR / "results.json"
CSV_PATH = RESULTS_DIR / "passage_scores.csv"

print(f"SEED: {SEED}")
print(f"Results directory: {RESULTS_DIR}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Load model
sys.path.insert(0, ".")

from lib.config import ExperimentConfig
from lib.model_utils import load_model

MODEL_NAME = "google/gemma-3-4b-it"

exp_config = ExperimentConfig(
    model_name=MODEL_NAME,
    model_type="gemma3",
    compute_dtype="auto",
    use_4bit=True,
    num_samples=2000,
    seed=SEED,
)

print(f"\nLoading {MODEL_NAME} (4-bit, bfloat16)...")
model, tokenizer = load_model(exp_config)

# Architecture diagnostics
from lib.kv_cache import (
    _get_text_config, _get_head_dim, _get_rope_theta_for_layer,
    _get_cache_keys, _ensure_dynamic_cache,
)

text_config = _get_text_config(model.config)
print(f"\nModel loaded successfully.")
print(f"  Model class: {type(model).__name__}")
print(f"  Text config class: {type(text_config).__name__}")
print(f"  Hidden size: {text_config.hidden_size}")
print(f"  Num layers: {text_config.num_hidden_layers}")
print(f"  Num attention heads: {text_config.num_attention_heads}")
print(f"  Num KV heads: {text_config.num_key_value_heads}")
print(f"  Head dim: {_get_head_dim(model.config)}")
print(f"  BOS token ID: {tokenizer.bos_token_id}")
print(f"  EOS token ID: {tokenizer.eos_token_id}")

# Per-layer RoPE diagnostics
thetas = set()
for layer_idx in range(text_config.num_hidden_layers):
    thetas.add(_get_rope_theta_for_layer(model.config, layer_idx))
print(f"  Unique RoPE thetas: {sorted(thetas)}")

# Verify dtype
sample_ids = tokenizer("test", return_tensors="pt")['input_ids'].to(exp_config.device)
with torch.no_grad():
    out = model(sample_ids, use_cache=True)
    cache_check = _ensure_dynamic_cache(out.past_key_values)
    k0 = _get_cache_keys(cache_check, 0)
    print(f"  Cache key dtype: {k0.dtype}")
    print(f"  Cache key shape: {k0.shape}  (batch, kv_heads, seq, head_dim)")
del out, sample_ids, cache_check
torch.cuda.empty_cache()

SEED: 42
Results directory: results/exp23
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
GPU memory: 42.3 GB


`torch_dtype` is deprecated! Use `dtype` instead!



Loading google/gemma-3-4b-it (4-bit, bfloat16)...


Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]


Model loaded successfully.
  Model class: Gemma3ForConditionalGeneration
  Text config class: Gemma3TextConfig
  Hidden size: 2560
  Num layers: 34
  Num attention heads: 8
  Num KV heads: 4
  Head dim: 256
  BOS token ID: 2
  EOS token ID: 1
  Unique RoPE thetas: [10000.0, 1000000.0]


  Cache key dtype: torch.bfloat16
  Cache key shape: torch.Size([1, 4, 2, 256])  (batch, kv_heads, seq, head_dim)


In [2]:
# Cell 2: Configuration
from lib.kv_cache import (
    _get_cache_keys,
    _get_cache_values,
    _set_cache_keys,
    _set_cache_values,
    _ensure_dynamic_cache,
    extract_and_truncate_cache_with_bos,
    correct_rope_positions_with_bos,
    score_answer_with_cache,
    deepcopy_cache,
    replace_values_at_layers,
    interpolate_values,
)
from lib.surrogate import STATIC_SURROGATE_QUERIES
from lib.data import count_words
from transformers import DynamicCache
from tqdm.auto import tqdm

N_QUERIES = 200
LAYER_CUTOFF = 17          # layers 0-16 inclusive
LAYER_CUTOFF_LOW = 9       # layers 0-8 inclusive (for oracle_vel_low)
CHECKPOINT_EVERY = 5
QVI_ALPHAS = [0.10, 0.25, 0.50]
INTERP_ALPHA = 0.25
MAX_PASSAGE_WORDS = 300

# 5 intent prefixes
INTENT_PREFIXES = {
    'fact': STATIC_SURROGATE_QUERIES['static_factual']['query'],
    'def': STATIC_SURROGATE_QUERIES['static_definitional']['query'],
    'proc': STATIC_SURROGATE_QUERIES['static_procedural']['query'],
    'quant': STATIC_SURROGATE_QUERIES['static_quantitative']['query'],
    'prob': STATIC_SURROGATE_QUERIES['static_problem']['query'],
}

# Scoring templates — 3 targets
ANSWER_QP = "\nQuery: {query}\nAnswer:"
ANSWER_AT = " {answer}"
QDOC_QP = "\nThis document is about:"
QDOC_AT = " {query}"
RELEVANCE_QP = "\nQuery: {query}\nIs this document relevant?"
RELEVANCE_AT = " Yes, this document is relevant to the query"

# All 13 condition names (for consistent ordering)
CONDITION_NAMES = [
    'bare', 'fact_vel', 'def_vel', 'proc_vel', 'quant_vel', 'prob_vel',
    'oracle_vel', 'oracle_vel_low', 'oracle_interp', 'oracle_full',
    'qvi_010', 'qvi_025', 'qvi_050',
]
TARGET_NAMES = ['answer', 'qdoc', 'relevance']

print("Configuration:")
print(f"  N_QUERIES: {N_QUERIES}")
print(f"  LAYER_CUTOFF: {LAYER_CUTOFF} (layers 0-{LAYER_CUTOFF-1})")
print(f"  LAYER_CUTOFF_LOW: {LAYER_CUTOFF_LOW} (layers 0-{LAYER_CUTOFF_LOW-1})")
print(f"  CHECKPOINT_EVERY: {CHECKPOINT_EVERY}")
print(f"  QVI_ALPHAS: {QVI_ALPHAS}")
print(f"  INTERP_ALPHA: {INTERP_ALPHA}")
print(f"  MAX_PASSAGE_WORDS: {MAX_PASSAGE_WORDS}")
print(f"\nIntent prefixes:")
for name, query in INTENT_PREFIXES.items():
    print(f"  {name}: '{query}'")
print(f"\nScoring targets:")
print(f"  answer:    QP='{ANSWER_QP}'  AT='{ANSWER_AT}'")
print(f"  qdoc:      QP='{QDOC_QP}'  AT='{QDOC_AT}'")
print(f"  relevance: QP='{RELEVANCE_QP}'  AT='{RELEVANCE_AT}'")
print(f"\n13 conditions × 3 targets = {len(CONDITION_NAMES) * len(TARGET_NAMES)} scores per passage")

Configuration:
  N_QUERIES: 200
  LAYER_CUTOFF: 17 (layers 0-16)
  LAYER_CUTOFF_LOW: 9 (layers 0-8)
  CHECKPOINT_EVERY: 5
  QVI_ALPHAS: [0.1, 0.25, 0.5]
  INTERP_ALPHA: 0.25
  MAX_PASSAGE_WORDS: 300

Intent prefixes:
  fact: 'What are the key facts I need to know?'
  def: 'What is this and what does it mean?'
  proc: 'How do I do this step by step?'
  quant: 'How much does this cost or how long does it take?'
  prob: 'What problem does this solve?'

Scoring targets:
  answer:    QP='
Query: {query}
Answer:'  AT=' {answer}'
  qdoc:      QP='
This document is about:'  AT=' {query}'
  relevance: QP='
Query: {query}
Is this document relevant?'  AT=' Yes, this document is relevant to the query'

13 conditions × 3 targets = 39 scores per passage


In [3]:
# Cell 3: Load MS MARCO Data
from datasets import load_dataset

print("=" * 70)
print("LOADING MS MARCO v1.1 — ALL PASSAGES PER QUERY")
print("=" * 70)

dataset = load_dataset("microsoft/ms_marco", "v1.1", split="validation",
                        trust_remote_code=True)
print(f"Total items in validation: {len(dataset)}")

queries = []
np.random.seed(SEED)

for item in tqdm(dataset, desc="Filtering"):
    passages_info = item.get('passages', {})
    passage_texts = passages_info.get('passage_text', [])
    is_selected = passages_info.get('is_selected', [])
    query = item.get('query', '')
    answers = item.get('answers', [])
    well_formed = item.get('wellFormedAnswers', [])

    if not passage_texts or not query:
        continue
    # Require at least 1 relevant passage
    if not is_selected or sum(is_selected) == 0:
        continue

    # All passages must be <= MAX_PASSAGE_WORDS
    word_counts = [count_words(p) for p in passage_texts]
    if any(wc > MAX_PASSAGE_WORDS for wc in word_counts):
        continue

    # Require valid answer
    answer = None
    if well_formed and len(well_formed) > 0 and well_formed[0] != '[]':
        answer = well_formed[0]
    elif answers and len(answers) > 0 and answers[0] != 'No Answer Present.':
        answer = answers[0]
    else:
        continue

    passage_list = []
    for i, (ptext, sel) in enumerate(zip(passage_texts, is_selected)):
        passage_list.append({
            'passage': ptext,
            'is_relevant': bool(sel == 1),
            'word_count': word_counts[i],
            'passage_idx': i,
        })

    queries.append({
        'query': query,
        'answer': answer,
        'passages': passage_list,
        'n_passages': len(passage_list),
        'n_relevant': sum(1 for p in passage_list if p['is_relevant']),
    })

    if len(queries) >= N_QUERIES * 3:
        break

np.random.shuffle(queries)
queries = queries[:N_QUERIES]
N = len(queries)

n_passages_list = [q['n_passages'] for q in queries]
total_passages = sum(n_passages_list)
total_relevant = sum(q['n_relevant'] for q in queries)
total_irrelevant = total_passages - total_relevant

print(f"\nSelected {N} queries ({total_passages} total passages)")
print(f"Passages per query: mean={np.mean(n_passages_list):.1f}, "
      f"min={min(n_passages_list)}, max={max(n_passages_list)}")
print(f"Word counts: mean={np.mean([p['word_count'] for q in queries for p in q['passages']]):.0f}")
print(f"Relevant passages: {total_relevant} ({100*total_relevant/total_passages:.1f}%)")
print(f"Irrelevant passages: {total_irrelevant} ({100*total_irrelevant/total_passages:.1f}%)")

del dataset
gc.collect()

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'microsoft/ms_marco' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


LOADING MS MARCO v1.1 — ALL PASSAGES PER QUERY


Total items in validation: 10047


Filtering:   0%|          | 0/10047 [00:00<?, ?it/s]


Selected 200 queries (1692 total passages)
Passages per query: mean=8.5, min=5, max=10
Word counts: mean=71
Relevant passages: 221 (13.1%)
Irrelevant passages: 1471 (86.9%)


52

In [4]:
# Cell 4: Explain Experimental Conditions
print("=" * 70)
print("EXPERIMENTAL CONDITIONS EXPLAINED")
print("=" * 70)

example_query = queries[0]['query']
example_answer = queries[0]['answer']
example_passage = queries[0]['passages'][0]['passage'][:80] + "..."

print(f"""
Example query:   \"{example_query}\"
Example answer:  \"{example_answer[:60]}...\"
Example passage: \"{example_passage}\"

=== 13 CACHE CONDITIONS ===

### 1. bare ###
  Cache: model([BOS][doc_ids])
  No prefix. Baseline cache.

### 2-6. {{intent}}_vel (5 intent-specialized VEL caches) ###
  For each intent in [fact, def, proc, quant, prob]:
    Step 1: Forward pass [BOS][intent_prefix][doc_ids]
    Step 2: Truncate to [BOS][last doc_len positions]
    Step 3: RoPE correct (offset = # prefix tokens)
    Step 4: replace_values_at_layers(bare, corrected, layers 0-{LAYER_CUTOFF-1})
  Prefixes:
    fact:  \"{INTENT_PREFIXES['fact']}\"
    def:   \"{INTENT_PREFIXES['def']}\"
    proc:  \"{INTENT_PREFIXES['proc']}\"
    quant: \"{INTENT_PREFIXES['quant']}\"
    prob:  \"{INTENT_PREFIXES['prob']}\"

### 7. oracle_vel ###
  Same as intent VEL, but prefix = actual query text.
  Layers 0-{LAYER_CUTOFF-1}. Upper bound for query-aware VEL.

### 8. oracle_vel_low ###
  Same as oracle_vel, but only layers 0-{LAYER_CUTOFF_LOW-1}.
  Reuses oracle's forward pass.

### 9. oracle_interp ###
  interpolate_values(bare, oracle_corrected, alpha={INTERP_ALPHA})
  v_blended = {INTERP_ALPHA} * v_oracle + {1-INTERP_ALPHA} * v_bare (ALL layers).
  Keys always from bare.

### 10. oracle_full ###
  Full [BOS][query][doc] cache, not truncated.
  context_len = 1 + prefix_len + doc_len.
  Gold standard: query is visible to scoring.

### 11-13. qvi_{{alpha}} (Query-Time Value Injection) ###
  For each alpha in {QVI_ALPHAS}:
    1. Build query cache: model([BOS][query_ids])
    2. Compute mean query values per layer: v_query_mean[layer]
    3. For layers 0-{LAYER_CUTOFF-1}: v_new = (1-alpha) * v_bare + alpha * v_query_mean
    4. Keys from bare.
  Formula: v_new = (1-α) * v_doc + α * mean(v_query)
  No extra forward pass — blends existing caches.

=== 3 SCORING TARGETS ===

### answer ###
  query_prompt: "\nQuery: {{query}}\nAnswer:"
  answer_text:  " {{answer}}"
  Standard: can the cache predict the actual answer?

### qdoc ###
  query_prompt: "\nThis document is about:"
  answer_text:  " {{query}}"
  Bidirectional: does the document predict the query?

### relevance ###
  query_prompt: "\nQuery: {{query}}\nIs this document relevant?"
  answer_text:  " Yes, this document is relevant to the query"
  Template: explicit relevance judgment (~8 tokens).

=== ANALYSIS-TIME DERIVED METHODS (no extra data) ===

1. PMI: score - baseline for all 39 condition × target pairs
2. Best-intent routing: min(fact, def, proc, quant, prob) per passage
3. Two-stage pipeline: bare PMI rank → re-rank top-k with oracle_vel PMI
4. Per-query intent routing: pick best intent per query
""")

# Verify relevance answer tokenizes to >= 2 tokens
rel_tokens = tokenizer(RELEVANCE_AT, add_special_tokens=False)['input_ids']
print(f"Relevance answer tokens ({len(rel_tokens)} tokens): {rel_tokens}")
print(f"Decoded: {[tokenizer.decode([t]) for t in rel_tokens]}")
assert len(rel_tokens) >= 2, f"Relevance answer must be >= 2 tokens, got {len(rel_tokens)}"
print(f"\nVerification passed: relevance answer = {len(rel_tokens)} tokens (>= 2).")

EXPERIMENTAL CONDITIONS EXPLAINED

Example query:   "what is the bride entrance called"
Example answer:  "Procession..."
Example passage: "Check out our ultimate guide below, beginning with Mom and ending with the bride..."

=== 13 CACHE CONDITIONS ===

### 1. bare ###
  Cache: model([BOS][doc_ids])
  No prefix. Baseline cache.

### 2-6. {intent}_vel (5 intent-specialized VEL caches) ###
  For each intent in [fact, def, proc, quant, prob]:
    Step 1: Forward pass [BOS][intent_prefix][doc_ids]
    Step 2: Truncate to [BOS][last doc_len positions]
    Step 3: RoPE correct (offset = # prefix tokens)
    Step 4: replace_values_at_layers(bare, corrected, layers 0-16)
  Prefixes:
    fact:  "What are the key facts I need to know?"
    def:   "What is this and what does it mean?"
    proc:  "How do I do this step by step?"
    quant: "How much does this cost or how long does it take?"
    prob:  "What problem does this solve?"

### 7. oracle_vel ###
  Same as intent VEL, but prefix = actual 

In [5]:
# Cell 5: Helper Functions

def build_query_cache(query_text, model, tokenizer, config):
    """Forward pass [BOS][query] → return (cache, len)."""
    enc = tokenizer(query_text, return_tensors="pt", add_special_tokens=True)
    ids = enc['input_ids'].to(config.device)
    with torch.no_grad():
        out = model(input_ids=ids, attention_mask=torch.ones_like(ids),
                    use_cache=True, return_dict=True)
    cache = _ensure_dynamic_cache(out.past_key_values)
    q_len = ids.shape[1]
    del out, ids, enc
    return cache, q_len


def build_qvi_cache(bare_cache, query_cache, layer_cutoff, alpha):
    """Blend mean query values into early layers of bare doc cache.

    Formula: v_new = (1-alpha) * v_bare + alpha * mean(v_query)
    Keys always from bare. Only affects layers < layer_cutoff.
    """
    new_cache = DynamicCache()
    for layer_idx in range(len(bare_cache)):
        k = _get_cache_keys(bare_cache, layer_idx).clone()
        v_bare = _get_cache_values(bare_cache, layer_idx)
        if layer_idx < layer_cutoff:
            v_query = _get_cache_values(query_cache, layer_idx)
            v_query_mean = v_query.mean(dim=2, keepdim=True)  # [1, heads, 1, dim]
            v_new = (1 - alpha) * v_bare + alpha * v_query_mean
            new_cache.update(k, v_new.clone(), layer_idx)
        else:
            new_cache.update(k, v_bare.clone(), layer_idx)
    return new_cache


def score_all_targets(cache, context_len, query, answer, model, tokenizer, config):
    """Score all 3 targets against one cache. Last target consumes cache (no deepcopy).

    Returns: {'answer': float, 'qdoc': float, 'relevance': float}
    """
    answer_qp = ANSWER_QP.format(query=query)
    answer_at = ANSWER_AT.format(answer=answer)
    qdoc_qp = QDOC_QP
    qdoc_at = QDOC_AT.format(query=query)
    rel_qp = RELEVANCE_QP.format(query=query)
    rel_at = RELEVANCE_AT

    nll_answer = score_answer_with_cache(
        deepcopy_cache(cache), context_len, answer_qp, answer_at,
        model, tokenizer, config)
    nll_qdoc = score_answer_with_cache(
        deepcopy_cache(cache), context_len, qdoc_qp, qdoc_at,
        model, tokenizer, config)
    nll_relevance = score_answer_with_cache(
        cache, context_len, rel_qp, rel_at,
        model, tokenizer, config)  # consumes cache — last use
    return {'answer': nll_answer, 'qdoc': nll_qdoc, 'relevance': nll_relevance}


def score_baselines(query, answer, model, tokenizer, config):
    """BOS-only cache → score all 3 targets. Returns dict of 3 baseline NLLs."""
    bos_id = torch.tensor([[tokenizer.bos_token_id]], device=config.device)
    with torch.no_grad():
        bos_out = model(input_ids=bos_id, attention_mask=torch.ones_like(bos_id),
                        use_cache=True, return_dict=True)
    bos_cache = _ensure_dynamic_cache(bos_out.past_key_values)
    del bos_out
    context_len = 1  # just BOS
    return score_all_targets(bos_cache, context_len, query, answer,
                             model, tokenizer, config)


def build_vel_cache(bare_cache, doc_len, prefix_str, passage_text,
                    model, tokenizer, config, layer_cutoff):
    """Build a VEL cache for a given prefix.

    Matched tokenization: tokenize prefix+passage together, split by prefix length.
    Returns VEL cache (bare keys, primed values in early layers).
    """
    full_text = prefix_str + passage_text
    full_enc = tokenizer(full_text, return_tensors="pt",
                          add_special_tokens=True, padding=False, truncation=False)
    full_ids = full_enc['input_ids'].to(config.device)

    sf_prefix_enc = tokenizer(prefix_str, return_tensors="pt",
                               add_special_tokens=True, padding=False, truncation=False)
    sf_prefix_len_matched = sf_prefix_enc['input_ids'].shape[1]
    del sf_prefix_enc

    # Forward pass: [BOS][prefix][doc]
    with torch.no_grad():
        primed_out = model(input_ids=full_ids,
                           attention_mask=torch.ones_like(full_ids),
                           use_cache=True, return_dict=True)
    primed_full = _ensure_dynamic_cache(primed_out.past_key_values)
    prefix_offset = sf_prefix_len_matched - 1  # tokens without BOS
    del primed_out, full_ids, full_enc

    # Truncate + RoPE correct
    trunc_cache = extract_and_truncate_cache_with_bos(primed_full, doc_len)
    del primed_full
    correct_rope_positions_with_bos(trunc_cache, prefix_offset, model)

    # Build VEL: bare keys, primed values in early layers
    vel_cache = replace_values_at_layers(
        bare_cache, trunc_cache, list(range(layer_cutoff)))
    del trunc_cache
    return vel_cache


def build_oracle_caches(bare_cache, doc_len, query, passage_text,
                        model, tokenizer, config):
    """Build all oracle-derived caches from one forward pass.

    Returns dict with keys: oracle_vel, oracle_vel_low, oracle_interp, oracle_full,
    plus oracle_full_context_len.
    """
    prefix_str = query + "\n"
    full_text = prefix_str + passage_text
    full_enc = tokenizer(full_text, return_tensors="pt",
                          add_special_tokens=True, padding=False, truncation=False)
    full_ids = full_enc['input_ids'].to(config.device)

    sf_prefix_enc = tokenizer(prefix_str, return_tensors="pt",
                               add_special_tokens=True, padding=False, truncation=False)
    sf_prefix_len_matched = sf_prefix_enc['input_ids'].shape[1]
    prefix_offset = sf_prefix_len_matched - 1  # tokens without BOS
    del sf_prefix_enc

    # Forward pass: [BOS][query][doc]
    with torch.no_grad():
        primed_out = model(input_ids=full_ids,
                           attention_mask=torch.ones_like(full_ids),
                           use_cache=True, return_dict=True)
    oracle_primed = _ensure_dynamic_cache(primed_out.past_key_values)
    oracle_full_context_len = full_ids.shape[1]  # 1 + prefix_len + doc_len
    del primed_out, full_ids, full_enc

    # --- oracle_full: score directly from primed cache (no truncation) ---
    oracle_full = deepcopy_cache(oracle_primed)

    # --- Truncate + RoPE correct for VEL variants ---
    oracle_trunc = extract_and_truncate_cache_with_bos(oracle_primed, doc_len)
    del oracle_primed
    correct_rope_positions_with_bos(oracle_trunc, prefix_offset, model)

    # oracle_vel: layers 0-16
    oracle_vel = replace_values_at_layers(
        bare_cache, oracle_trunc, list(range(LAYER_CUTOFF)))

    # oracle_vel_low: layers 0-8
    oracle_vel_low = replace_values_at_layers(
        bare_cache, oracle_trunc, list(range(LAYER_CUTOFF_LOW)))

    # oracle_interp: interpolate ALL layers
    oracle_interp = interpolate_values(bare_cache, oracle_trunc, INTERP_ALPHA)

    del oracle_trunc

    return {
        'oracle_vel': oracle_vel,
        'oracle_vel_low': oracle_vel_low,
        'oracle_interp': oracle_interp,
        'oracle_full': oracle_full,
        'oracle_full_context_len': oracle_full_context_len,
    }


def build_and_score_all_conditions(passage_text, query, answer, query_cache,
                                    model, tokenizer, config):
    """Build all 13 caches and score all 3 targets for one passage.

    Memory-safe processing order: build, score, delete sequentially.
    Returns: dict of {condition_name: {target: nll}} plus doc_len.
    """
    scores = {}

    # === 1. Build bare cache (KEEP alive throughout) ===
    bare_enc = tokenizer(passage_text, return_tensors="pt",
                          add_special_tokens=True, padding=False, truncation=False)
    bare_ids = bare_enc['input_ids'].to(config.device)
    doc_len = bare_ids.shape[1] - 1  # exclude BOS
    context_len = 1 + doc_len  # BOS + doc

    with torch.no_grad():
        bare_out = model(input_ids=bare_ids,
                         attention_mask=torch.ones_like(bare_ids),
                         use_cache=True, return_dict=True)
    bare_cache = _ensure_dynamic_cache(bare_out.past_key_values)
    del bare_out, bare_ids, bare_enc

    # === 2. Score bare ===
    scores['bare'] = score_all_targets(
        deepcopy_cache(bare_cache), context_len, query, answer,
        model, tokenizer, config)

    # === 3. Five intent VEL caches ===
    intent_to_cond = {
        'fact': 'fact_vel', 'def': 'def_vel', 'proc': 'proc_vel',
        'quant': 'quant_vel', 'prob': 'prob_vel',
    }
    for intent_name, cond_name in intent_to_cond.items():
        prefix_str = INTENT_PREFIXES[intent_name] + "\n"
        vel = build_vel_cache(bare_cache, doc_len, prefix_str, passage_text,
                              model, tokenizer, config, LAYER_CUTOFF)
        scores[cond_name] = score_all_targets(
            vel, context_len, query, answer, model, tokenizer, config)
        del vel

    # === 4. Oracle caches (one FP, four conditions) ===
    oracle = build_oracle_caches(bare_cache, doc_len, query, passage_text,
                                  model, tokenizer, config)

    # oracle_vel (L0-16)
    scores['oracle_vel'] = score_all_targets(
        oracle['oracle_vel'], context_len, query, answer,
        model, tokenizer, config)
    del oracle['oracle_vel']

    # oracle_vel_low (L0-8)
    scores['oracle_vel_low'] = score_all_targets(
        oracle['oracle_vel_low'], context_len, query, answer,
        model, tokenizer, config)
    del oracle['oracle_vel_low']

    # oracle_interp
    scores['oracle_interp'] = score_all_targets(
        oracle['oracle_interp'], context_len, query, answer,
        model, tokenizer, config)
    del oracle['oracle_interp']

    # oracle_full (different context_len!)
    scores['oracle_full'] = score_all_targets(
        oracle['oracle_full'], oracle['oracle_full_context_len'],
        query, answer, model, tokenizer, config)
    del oracle

    # === 5. QVI caches ===
    for alpha in QVI_ALPHAS:
        cond_name = f'qvi_{int(alpha*100):03d}'
        qvi = build_qvi_cache(bare_cache, query_cache, LAYER_CUTOFF, alpha)
        scores[cond_name] = score_all_targets(
            qvi, context_len, query, answer, model, tokenizer, config)
        del qvi

    # === 6. Cleanup ===
    del bare_cache
    torch.cuda.empty_cache()
    gc.collect()

    return scores, doc_len


# === Smoke test ===
print("Smoke-testing helper functions...")
test_q = queries[0]
test_query = test_q['query']
test_answer = test_q['answer']
test_passage = test_q['passages'][0]['passage']

# Test baselines (3 targets)
test_baselines = score_baselines(test_query, test_answer, model, tokenizer, exp_config)
print(f"  Baselines: {test_baselines}")
for tgt, nll in test_baselines.items():
    assert np.isfinite(nll), f"Baseline {tgt} NLL not finite: {nll}"

# Test query cache
test_qcache, test_qlen = build_query_cache(test_query, model, tokenizer, exp_config)
print(f"  Query cache built: {test_qlen} tokens")

# Test full passage scoring (13 conditions × 3 targets)
test_scores, test_doc_len = build_and_score_all_conditions(
    test_passage, test_query, test_answer, test_qcache,
    model, tokenizer, exp_config)
print(f"  Doc len: {test_doc_len}")
print(f"  Conditions scored: {len(test_scores)}")
n_finite = 0
for cond, targets in test_scores.items():
    for tgt, nll in targets.items():
        assert np.isfinite(nll), f"{cond}/{tgt} NLL not finite: {nll}"
        n_finite += 1
print(f"  All {n_finite} NLL values are finite.")

# Print a summary
print(f"\n  {'Condition':<20} {'answer':>10} {'qdoc':>10} {'relevance':>10}")
print("  " + "-" * 52)
for cond in CONDITION_NAMES:
    t = test_scores[cond]
    print(f"  {cond:<20} {t['answer']:>10.4f} {t['qdoc']:>10.4f} {t['relevance']:>10.4f}")

del test_qcache, test_scores
torch.cuda.empty_cache()
gc.collect()
print(f"\nSmoke test passed: 13 conditions × 3 targets = {n_finite} scores, all finite.")

Smoke-testing helper functions...


  Baselines: {'answer': 4.75, 'qdoc': 8.45, 'relevance': 1.3359375}
  Query cache built: 7 tokens


  Doc len: 151
  Conditions scored: 13
  All 39 NLL values are finite.

  Condition                answer       qdoc  relevance
  ----------------------------------------------------
  bare                     8.1250     5.1000     1.8750
  fact_vel                 9.6250     4.8000     1.2891
  def_vel                  8.7500     4.9000     1.2656
  proc_vel                 8.8750     4.9250     1.2656
  quant_vel                8.0000     4.8500     1.2344
  prob_vel                 8.6250     4.8750     1.1172
  oracle_vel               8.2500     4.6500     1.2578
  oracle_vel_low           8.5000     5.1000     1.6719
  oracle_interp            8.5000     4.9500     1.7266
  oracle_full              9.2500     0.0106     1.1328
  qvi_010                  6.8750     5.2500     1.5312
  qvi_025                  5.3750     5.6500     1.1250
  qvi_050                  3.1719     5.9500     1.2500



Smoke test passed: 13 conditions × 3 targets = 39 scores, all finite.


In [6]:
# Cell 6: Evaluation Loop

print("=" * 70)
print(f"MAIN EVALUATION ({N} queries, ~{total_passages} passages)")
print("Model: Gemma 3 4B | 13 conditions × 3 targets = 39 scores/passage")
print("=" * 70)

# Checkpoint resume
all_results = []
start_idx = 0

if CHECKPOINT_PATH.exists():
    with open(CHECKPOINT_PATH, 'r') as f:
        ckpt = json.load(f)
    ckpt_queries = ckpt.get('query_texts', [])
    current_queries = [q['query'] for q in queries]
    if ckpt_queries == current_queries:
        all_results = ckpt['results']
        start_idx = len(all_results)
        print(f"Resuming from checkpoint: {start_idx}/{N}")
    else:
        print("Checkpoint query mismatch. Starting fresh.")
else:
    print("No checkpoint found. Starting fresh.")

print(f"Evaluating queries {start_idx} to {N-1}")
print(f"Per passage: 7 forward passes + 39 scoring calls")
print(f"Per query: +1 query FP + 3 baseline scores")

t_start = time.time()

for qidx in tqdm(range(start_idx, N), initial=start_idx, total=N, desc="Queries"):
    query_data = queries[qidx]
    query = query_data['query']
    answer = query_data['answer']

    # 1. Build query cache for QVI (once per query)
    query_cache, query_len = build_query_cache(query, model, tokenizer, exp_config)

    # 2. Compute baselines for all 3 targets (once per query)
    baselines = score_baselines(query, answer, model, tokenizer, exp_config)

    # 3. Score each passage (all 13 conditions × 3 targets)
    passage_results = []
    for pidx, pinfo in enumerate(query_data['passages']):
        scores, doc_len = build_and_score_all_conditions(
            pinfo['passage'], query, answer, query_cache,
            model, tokenizer, exp_config)

        # Flatten scores into per-passage dict
        passage_row = {
            'passage_idx': pinfo['passage_idx'],
            'is_relevant': pinfo['is_relevant'],
            'word_count': pinfo['word_count'],
            'doc_len': doc_len,
        }
        for cond in CONDITION_NAMES:
            for tgt in TARGET_NAMES:
                passage_row[f'{cond}_{tgt}'] = scores[cond][tgt]
        passage_results.append(passage_row)

    # 4. Store query result
    all_results.append({
        'query_idx': qidx,
        'query': query,
        'answer': answer,
        'n_passages': len(passage_results),
        'n_relevant': query_data['n_relevant'],
        'baselines': baselines,
        'passage_data': passage_results,
    })

    # 5. Cleanup query cache
    del query_cache
    torch.cuda.empty_cache()
    gc.collect()

    # 6. Checkpoint
    if (qidx + 1) % CHECKPOINT_EVERY == 0 or qidx == N - 1:
        ckpt_data = {
            'results': all_results,
            'query_texts': [q['query'] for q in queries],
            'completed': len(all_results),
            'total': N,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        }
        with open(CHECKPOINT_PATH, 'w') as f:
            json.dump(ckpt_data, f)
        elapsed = time.time() - t_start
        n_done = qidx - start_idx + 1
        rate = n_done / elapsed if elapsed > 0 else 0
        remaining = (N - qidx - 1) / rate if rate > 0 else 0
        gpu_mem = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0
        tqdm.write(f"  Checkpoint {qidx+1}/{N} | {n_done} done in {elapsed/60:.1f}m | "
                   f"ETA: {remaining/60:.1f} min | GPU mem: {gpu_mem:.2f} GB")

elapsed_total = time.time() - t_start
print(f"\nEvaluation complete: {len(all_results)} queries in {elapsed_total/60:.1f} min")

MAIN EVALUATION (200 queries, ~1692 passages)
Model: Gemma 3 4B | 13 conditions × 3 targets = 39 scores/passage
No checkpoint found. Starting fresh.
Evaluating queries 0 to 199
Per passage: 7 forward passes + 39 scoring calls
Per query: +1 query FP + 3 baseline scores


Queries:   0%|          | 0/200 [00:00<?, ?it/s]

  Checkpoint 5/200 | 5 done in 8.7m | ETA: 339.6 min | GPU mem: 3.25 GB


  Checkpoint 10/200 | 10 done in 17.5m | ETA: 333.4 min | GPU mem: 3.25 GB


  Checkpoint 15/200 | 15 done in 26.2m | ETA: 323.3 min | GPU mem: 3.25 GB


...

  Checkpoint 180/200 | 180 done in 321.1m | ETA: 35.7 min | GPU mem: 3.25 GB


  Checkpoint 185/200 | 185 done in 330.3m | ETA: 26.8 min | GPU mem: 3.25 GB


  Checkpoint 190/200 | 190 done in 338.8m | ETA: 17.8 min | GPU mem: 3.25 GB


  Checkpoint 195/200 | 195 done in 348.4m | ETA: 8.9 min | GPU mem: 3.25 GB


  Checkpoint 200/200 | 200 done in 357.4m | ETA: 0.0 min | GPU mem: 3.25 GB

Evaluation complete: 200 queries in 357.4 min


In [7]:
# Cell 7: Analysis
from sklearn.metrics import roc_auc_score, roc_curve
from scipy import stats

print("=" * 70)
print("ANALYSIS — MULTI-SIGNAL RANKING")
print("=" * 70)

# --- Flatten passage-level data ---
is_relevant_all = []
nll_data = {f'{c}_{t}': [] for c in CONDITION_NAMES for t in TARGET_NAMES}
baseline_data = {t: [] for t in TARGET_NAMES}

for r in all_results:
    bl = r['baselines']
    for p in r['passage_data']:
        is_relevant_all.append(int(p['is_relevant']))
        for c in CONDITION_NAMES:
            for t in TARGET_NAMES:
                nll_data[f'{c}_{t}'].append(p[f'{c}_{t}'])
        for t in TARGET_NAMES:
            baseline_data[t].append(bl[t])

is_relevant = np.array(is_relevant_all)
n_total = len(is_relevant)
n_rel = int(is_relevant.sum())
n_irr = n_total - n_rel

# Convert to numpy
nll_arrays = {k: np.array(v) for k, v in nll_data.items()}
baseline_arrays = {k: np.array(v) for k, v in baseline_data.items()}

print(f"\nTotal passages: {n_total}")
print(f"Relevant: {n_rel} ({100*n_rel/n_total:.1f}%)")
print(f"Irrelevant: {n_irr} ({100*n_irr/n_total:.1f}%)")

# === Compute AUC for all 13×3 raw + PMI ===
print("\n" + "=" * 70)
print("AUC-ROC: RAW NLL (lower = more relevant → negate for AUC)")
print("=" * 70)

auc_raw = {}
auc_pmi = {}

print(f"\n{'Condition':<20}", end="")
for t in TARGET_NAMES:
    print(f" {'Raw '+t:>12} {'PMI '+t:>12}", end="")
print()
print("-" * 92)

for c in CONDITION_NAMES:
    print(f"{c:<20}", end="")
    for t in TARGET_NAMES:
        key = f'{c}_{t}'
        raw_scores = nll_arrays[key]
        pmi_scores = raw_scores - baseline_arrays[t]
        auc_r = roc_auc_score(is_relevant, -raw_scores)
        auc_p = roc_auc_score(is_relevant, -pmi_scores)
        auc_raw[key] = auc_r
        auc_pmi[key] = auc_p
        print(f" {auc_r:>12.3f} {auc_p:>12.3f}", end="")
    print()

# === MRR@10 ===
print("\n" + "=" * 70)
print("MRR@10")
print("=" * 70)

def compute_mrr_at_k(all_results, score_key, baseline_key=None, k=10):
    """Compute MRR@k. If baseline_key, use PMI (score - baseline)."""
    rr_list = []
    for r in all_results:
        passages = r['passage_data']
        bl = r['baselines']
        scored = []
        for p in passages:
            s = p[score_key]
            if baseline_key:
                s = s - bl[baseline_key]
            scored.append((s, p['is_relevant']))
        scored.sort(key=lambda x: x[0])  # lower = more relevant
        rr = 0.0
        for rank, (score, rel) in enumerate(scored[:k], 1):
            if rel:
                rr = 1.0 / rank
                break
        rr_list.append(rr)
    return np.mean(rr_list), rr_list

mrr_raw = {}
mrr_pmi = {}

print(f"\n{'Condition':<20}", end="")
for t in TARGET_NAMES:
    print(f" {'Raw '+t:>12} {'PMI '+t:>12}", end="")
print()
print("-" * 92)

for c in CONDITION_NAMES:
    print(f"{c:<20}", end="")
    for t in TARGET_NAMES:
        key = f'{c}_{t}'
        mrr_r, _ = compute_mrr_at_k(all_results, key, k=10)
        mrr_p, _ = compute_mrr_at_k(all_results, key, baseline_key=t, k=10)
        mrr_raw[key] = mrr_r
        mrr_pmi[key] = mrr_p
        print(f" {mrr_r:>12.3f} {mrr_p:>12.3f}", end="")
    print()

# === Derived methods ===
print("\n" + "=" * 70)
print("DERIVED METHODS (analysis-time, no extra data)")
print("=" * 70)

intent_conds = ['fact_vel', 'def_vel', 'proc_vel', 'quant_vel', 'prob_vel']

# 1. Best-intent routing: min across 5 intents per passage
print("\n--- Best-Intent Routing (min across 5 intents per passage) ---")
print(f"{'Target':<12} {'AUC Raw':>10} {'AUC PMI':>10} {'MRR Raw':>10} {'MRR PMI':>10}")
print("-" * 55)

derived_auc = {}
derived_mrr = {}

for t in TARGET_NAMES:
    # Best-intent routing (raw): min NLL across intents per passage
    intent_scores = np.stack([nll_arrays[f'{c}_{t}'] for c in intent_conds], axis=1)
    best_raw = intent_scores.min(axis=1)
    best_pmi = (intent_scores - baseline_arrays[t][:, None]).min(axis=1)
    auc_r = roc_auc_score(is_relevant, -best_raw)
    auc_p = roc_auc_score(is_relevant, -best_pmi)
    derived_auc[f'best_intent_{t}_raw'] = auc_r
    derived_auc[f'best_intent_{t}_pmi'] = auc_p

    # MRR for best-intent: compute per-query
    rr_raw_list, rr_pmi_list = [], []
    for r in all_results:
        bl = r['baselines'][t]
        scored_raw, scored_pmi = [], []
        for p in r['passage_data']:
            vals = [p[f'{c}_{t}'] for c in intent_conds]
            scored_raw.append((min(vals), p['is_relevant']))
            scored_pmi.append((min(v - bl for v in vals), p['is_relevant']))
        for scored, rr_list in [(scored_raw, rr_raw_list), (scored_pmi, rr_pmi_list)]:
            scored.sort(key=lambda x: x[0])
            rr = 0.0
            for rank, (s, rel) in enumerate(scored[:10], 1):
                if rel:
                    rr = 1.0 / rank
                    break
            rr_list.append(rr)
    mrr_r = np.mean(rr_raw_list)
    mrr_p = np.mean(rr_pmi_list)
    derived_mrr[f'best_intent_{t}_raw'] = mrr_r
    derived_mrr[f'best_intent_{t}_pmi'] = mrr_p
    print(f"{t:<12} {auc_r:>10.3f} {auc_p:>10.3f} {mrr_r:>10.3f} {mrr_p:>10.3f}")

# 2. Per-query intent routing: pick best intent per query (by mean NLL)
print("\n--- Per-Query Intent Routing (pick best intent per query) ---")
print(f"{'Target':<12} {'AUC Raw':>10} {'AUC PMI':>10} {'MRR Raw':>10} {'MRR PMI':>10}")
print("-" * 55)

for t in TARGET_NAMES:
    # For each query, find which intent has lowest mean NLL
    routed_raw = []
    routed_pmi = []
    rr_raw_list, rr_pmi_list = [], []

    for r in all_results:
        bl = r['baselines'][t]
        # Mean NLL per intent for this query
        mean_nlls = {}
        for c in intent_conds:
            mean_nlls[c] = np.mean([p[f'{c}_{t}'] for p in r['passage_data']])
        best_intent = min(mean_nlls, key=mean_nlls.get)

        scored_raw, scored_pmi = [], []
        for p in r['passage_data']:
            s = p[f'{best_intent}_{t}']
            routed_raw.append(s)
            routed_pmi.append(s - bl)
            scored_raw.append((s, p['is_relevant']))
            scored_pmi.append((s - bl, p['is_relevant']))

        for scored, rr_list in [(scored_raw, rr_raw_list), (scored_pmi, rr_pmi_list)]:
            scored.sort(key=lambda x: x[0])
            rr = 0.0
            for rank, (s, rel) in enumerate(scored[:10], 1):
                if rel:
                    rr = 1.0 / rank
                    break
            rr_list.append(rr)

    routed_raw = np.array(routed_raw)
    routed_pmi = np.array(routed_pmi)
    auc_r = roc_auc_score(is_relevant, -routed_raw)
    auc_p = roc_auc_score(is_relevant, -routed_pmi)
    mrr_r = np.mean(rr_raw_list)
    mrr_p = np.mean(rr_pmi_list)
    derived_auc[f'query_route_{t}_raw'] = auc_r
    derived_auc[f'query_route_{t}_pmi'] = auc_p
    derived_mrr[f'query_route_{t}_raw'] = mrr_r
    derived_mrr[f'query_route_{t}_pmi'] = mrr_p
    print(f"{t:<12} {auc_r:>10.3f} {auc_p:>10.3f} {mrr_r:>10.3f} {mrr_p:>10.3f}")

# 3. Two-stage pipeline: bare PMI → re-rank top-k with oracle_vel PMI
print("\n--- Two-Stage Pipeline: Bare PMI → Oracle VEL Re-rank ---")
print(f"{'Target':<12} {'k':>4} {'MRR@10':>10}")
print("-" * 30)

two_stage_mrr = {}
for t in TARGET_NAMES:
    for k in [3, 5]:
        rr_list = []
        for r in all_results:
            bl = r['baselines'][t]
            # Stage 1: rank by bare PMI
            bare_scored = []
            for p in r['passage_data']:
                bare_pmi = p[f'bare_{t}'] - bl
                oracle_pmi = p[f'oracle_vel_{t}'] - bl
                bare_scored.append((bare_pmi, oracle_pmi, p['is_relevant']))
            bare_scored.sort(key=lambda x: x[0])  # lower = more relevant

            # Stage 2: re-rank top-k by oracle_vel PMI
            top_k = bare_scored[:k]
            rest = bare_scored[k:]
            top_k.sort(key=lambda x: x[1])  # re-rank by oracle PMI
            reranked = top_k + rest

            rr = 0.0
            for rank, (_, _, rel) in enumerate(reranked[:10], 1):
                if rel:
                    rr = 1.0 / rank
                    break
            rr_list.append(rr)
        mrr = np.mean(rr_list)
        two_stage_mrr[f'{t}_k{k}'] = mrr
        print(f"{t:<12} {k:>4} {mrr:>10.3f}")

# === Summary: Best method per target ===
print("\n" + "=" * 70)
print("BEST METHODS PER TARGET (PMI AUC)")
print("=" * 70)

for t in TARGET_NAMES:
    # Gather all AUC PMI values for this target
    target_aucs = {c: auc_pmi[f'{c}_{t}'] for c in CONDITION_NAMES}
    # Add derived
    for key in ['best_intent', 'query_route']:
        if f'{key}_{t}_pmi' in derived_auc:
            target_aucs[key] = derived_auc[f'{key}_{t}_pmi']
    best_name = max(target_aucs, key=target_aucs.get)
    print(f"\n  {t}: best = {best_name} (AUC = {target_aucs[best_name]:.3f})")
    sorted_aucs = sorted(target_aucs.items(), key=lambda x: x[1], reverse=True)
    for name, auc in sorted_aucs[:5]:
        print(f"    {name:<20} {auc:.3f}")

# === Head-to-head: each condition vs bare ===
print("\n" + "=" * 70)
print("HEAD-TO-HEAD vs BARE (per-query MRR wins/ties/losses, PMI answer target)")
print("=" * 70)

print(f"\n{'Condition':<20} {'Wins':>6} {'Ties':>6} {'Losses':>6}")
print("-" * 42)

t = 'answer'  # primary target
bare_rr = []
cond_rr = {c: [] for c in CONDITION_NAMES if c != 'bare'}

for r in all_results:
    bl = r['baselines'][t]
    # Bare RR
    scored = [(p[f'bare_{t}'] - bl, p['is_relevant']) for p in r['passage_data']]
    scored.sort(key=lambda x: x[0])
    rr = 0.0
    for rank, (s, rel) in enumerate(scored[:10], 1):
        if rel:
            rr = 1.0 / rank
            break
    bare_rr.append(rr)

    for c in cond_rr:
        scored = [(p[f'{c}_{t}'] - bl, p['is_relevant']) for p in r['passage_data']]
        scored.sort(key=lambda x: x[0])
        rr = 0.0
        for rank, (s, rel) in enumerate(scored[:10], 1):
            if rel:
                rr = 1.0 / rank
                break
        cond_rr[c].append(rr)

bare_rr = np.array(bare_rr)
for c in CONDITION_NAMES:
    if c == 'bare':
        continue
    c_rr = np.array(cond_rr[c])
    wins = int(np.sum(c_rr > bare_rr))
    ties = int(np.sum(c_rr == bare_rr))
    losses = int(np.sum(c_rr < bare_rr))
    print(f"{c:<20} {wins:>6} {ties:>6} {losses:>6}")

ANALYSIS — MULTI-SIGNAL RANKING

Total passages: 1692
Relevant: 221 (13.1%)
Irrelevant: 1471 (86.9%)

AUC-ROC: RAW NLL (lower = more relevant → negate for AUC)

Condition              Raw answer   PMI answer     Raw qdoc     PMI qdoc Raw relevance PMI relevance
--------------------------------------------------------------------------------------------
bare                        0.828        0.841        0.570        0.574        0.448        0.451
fact_vel                    0.829        0.832        0.569        0.570        0.455        0.454
def_vel                     0.827        0.835        0.567        0.568        0.455        0.456
proc_vel                    0.830        0.834        0.567        0.568        0.457        0.462
quant_vel                   0.831        0.830        0.566        0.565        0.462        0.466
prob_vel                    0.829        0.830        0.569        0.569        0.456        0.459
oracle_vel                  0.828        0.831     

        0.502        0.497        0.457        0.455
qvi_010                     0.822        0.836        0.560        0.563        0.469        0.471
qvi_025                     0.794        0.811        0.553        0.558        0.488        0.488
qvi_050                     0.733        0.767        0.550        0.555

        0.500        0.498

MRR@10

Condition              Raw answer   PMI answer     Raw qdoc     PMI qdoc Raw relevance PMI relevance
--------------------------------------------------------------------------------------------
bare                        0.860        0.860        0.436        0.436        0.298        0.298
fact_vel                    0.853        0.853        0.462        0.462        0.306        0.306
def_vel                     0.853        0.853        0.462        0.462        0.311        0.311
proc_vel                    0.862        0.862        0.453        0.453        0.307        0.307
quant_vel                   0.863        0.863        0.458        0.458        0.310        0.310
prob_vel                    0.859        0.859        0.447        0.447        0.313        0.313
oracle_vel                  0.865        0.865        0.459        0.459        0.286        0.286
oracle_vel_low              0.864        0.864        0.447        0.447     

qdoc              0.567      0.567      0.465      0.465
relevance         0.447      0.453      0.295      0.295

--- Two-Stage Pipeline: Bare PMI → Oracle VEL Re-rank ---
Target          k     MRR@10
------------------------------
answer          3      0.866
answer          5      0.865
qdoc            3      0.443
qdoc            5      0.460
relevance       3      0.298
relevance       5      0.292

BEST METHODS PER TARGET (PMI AUC)

  answer: best = oracle_interp (AUC = 0.842)
    oracle_interp        0.842
    bare                 0.841
    oracle_vel_low       0.841
    oracle_full          0.837
    qvi_010              0.836

  qdoc: best = oracle_interp (AUC = 0.574)
    oracle_interp        0.574
    bare                 0.574
    oracle_vel_low       0.573
    fact_vel             0.570
    prob_vel             0.569

  relevance: best = qvi_050 (AUC = 0.498)
    qvi_050              0.498
    qvi_025              0.488
    qvi_010              0.471
    quant_vel         

fact_vel                  6    185      9
def_vel                   6    184     10
proc_vel                  6    189      5
quant_vel                 6    188      6
prob_vel                  7    186      7
oracle_vel                8    187      5
oracle_vel_low            8    189      3
oracle_interp             2    197      1
oracle_full               7    185      8
qvi_010                   6    183     11
qvi_025                   7    176     17
qvi_050                   9    172     19


In [8]:
# Cell 8: Figures
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

fig_dir = RESULTS_DIR / 'figures'

# === Figure 1: Heatmap — 13 conditions × 3 targets (AUC PMI) ===
fig1, ax1 = plt.subplots(figsize=(8, 7))
heatmap_data = np.zeros((len(CONDITION_NAMES), len(TARGET_NAMES)))
for i, c in enumerate(CONDITION_NAMES):
    for j, t in enumerate(TARGET_NAMES):
        heatmap_data[i, j] = auc_pmi[f'{c}_{t}']

im = ax1.imshow(heatmap_data, cmap='YlOrRd', aspect='auto', vmin=0.5, vmax=1.0)
ax1.set_xticks(range(len(TARGET_NAMES)))
ax1.set_xticklabels(TARGET_NAMES, fontsize=10)
ax1.set_yticks(range(len(CONDITION_NAMES)))
ax1.set_yticklabels(CONDITION_NAMES, fontsize=9)
ax1.set_title('AUC-ROC (PMI) — 13 Conditions × 3 Targets', fontsize=12)
for i in range(len(CONDITION_NAMES)):
    for j in range(len(TARGET_NAMES)):
        ax1.text(j, i, f'{heatmap_data[i, j]:.3f}', ha='center', va='center',
                fontsize=8, color='black' if heatmap_data[i, j] < 0.85 else 'white')
plt.colorbar(im, ax=ax1, label='AUC')
fig1.tight_layout()
fig1.savefig(fig_dir / 'heatmap.png', dpi=150, bbox_inches='tight')
plt.close(fig1)
print("Saved heatmap.png")

# === Figure 2: Bar chart — AUC for all conditions under `answer` target ===
fig2, ax2 = plt.subplots(figsize=(12, 5))
t = 'answer'
aucs_answer = [auc_pmi[f'{c}_{t}'] for c in CONDITION_NAMES]
colors_bar = ['#1f77b4'] + ['#2ca02c']*5 + ['#ff7f0e']*4 + ['#d62728']*3
bars = ax2.bar(range(len(CONDITION_NAMES)), aucs_answer, color=colors_bar, alpha=0.8)
ax2.set_xticks(range(len(CONDITION_NAMES)))
ax2.set_xticklabels(CONDITION_NAMES, rotation=45, ha='right', fontsize=9)
ax2.set_ylabel('AUC (PMI)')
ax2.set_title('AUC-ROC (PMI, answer target) by Condition')
ax2.axhline(y=auc_pmi[f'bare_{t}'], color='gray', linestyle='--', alpha=0.7, label='bare')
ax2.legend(fontsize=9)
ax2.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, aucs_answer):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002,
            f'{val:.3f}', ha='center', va='bottom', fontsize=7)
fig2.tight_layout()
fig2.savefig(fig_dir / 'auc_answer.png', dpi=150, bbox_inches='tight')
plt.close(fig2)
print("Saved auc_answer.png")

# === Figure 3: QVI alpha sensitivity curve ===
fig3, ax3 = plt.subplots(figsize=(7, 5))
for t in TARGET_NAMES:
    alphas = QVI_ALPHAS
    aucs_qvi = [auc_pmi[f'qvi_{int(a*100):03d}_{t}'] for a in alphas]
    ax3.plot(alphas, aucs_qvi, 'o-', label=f'{t}', linewidth=2, markersize=8)
    # Add bare baseline
    ax3.axhline(y=auc_pmi[f'bare_{t}'], linestyle='--', alpha=0.3)
ax3.set_xlabel('QVI Alpha')
ax3.set_ylabel('AUC (PMI)')
ax3.set_title('QVI: AUC vs Blending Alpha')
ax3.legend(fontsize=9)
ax3.grid(alpha=0.3)
fig3.tight_layout()
fig3.savefig(fig_dir / 'qvi_alpha.png', dpi=150, bbox_inches='tight')
plt.close(fig3)
print("Saved qvi_alpha.png")

# === Figure 4: Two-stage pipeline MRR ===
fig4, ax4 = plt.subplots(figsize=(7, 5))
for t in TARGET_NAMES:
    ks = [3, 5]
    mrrs_ts = [two_stage_mrr[f'{t}_k{k}'] for k in ks]
    ax4.plot(ks, mrrs_ts, 'o-', label=f'{t}', linewidth=2, markersize=8)
    # Bare PMI MRR baseline
    bare_mrr_val = mrr_pmi[f'bare_{t}']
    ax4.axhline(y=bare_mrr_val, linestyle='--', alpha=0.3)
ax4.set_xlabel('Top-k cutoff')
ax4.set_ylabel('MRR@10')
ax4.set_title('Two-Stage Pipeline: Bare PMI → Oracle VEL Re-rank')
ax4.legend(fontsize=9)
ax4.grid(alpha=0.3)
ax4.set_xticks(ks)
fig4.tight_layout()
fig4.savefig(fig_dir / 'two_stage.png', dpi=150, bbox_inches='tight')
plt.close(fig4)
print("Saved two_stage.png")

# === Figure 5: Score distributions — best condition vs bare (violin plots) ===
fig5, axes5 = plt.subplots(1, 3, figsize=(15, 5))
for idx, t in enumerate(TARGET_NAMES):
    ax = axes5[idx]
    # Find best non-bare condition for this target (PMI AUC)
    best_cond = max(
        [c for c in CONDITION_NAMES if c != 'bare'],
        key=lambda c: auc_pmi[f'{c}_{t}']
    )
    bare_pmi = nll_arrays[f'bare_{t}'] - baseline_arrays[t]
    best_pmi = nll_arrays[f'{best_cond}_{t}'] - baseline_arrays[t]

    data = [
        bare_pmi[is_relevant == 1], bare_pmi[is_relevant == 0],
        best_pmi[is_relevant == 1], best_pmi[is_relevant == 0],
    ]
    labels = ['bare\nRel', 'bare\nIrr', f'{best_cond}\nRel', f'{best_cond}\nIrr']
    vp = ax.violinplot(data, positions=[1, 2, 3, 4], showmeans=True, showmedians=True)
    ax.set_xticks([1, 2, 3, 4])
    ax.set_xticklabels(labels, fontsize=8)
    ax.set_ylabel('PMI Score')
    ax.set_title(f'{t} (best: {best_cond})', fontsize=10)
    ax.grid(axis='y', alpha=0.3)
fig5.suptitle('PMI Score Distributions: Best Condition vs Bare', fontsize=12)
fig5.tight_layout()
fig5.savefig(fig_dir / 'score_distributions.png', dpi=150, bbox_inches='tight')
plt.close(fig5)
print("Saved score_distributions.png")

print(f"\nAll figures saved to {fig_dir}")

Saved heatmap.png


Saved auc_answer.png


Saved qvi_alpha.png


Saved two_stage.png


Saved score_distributions.png

All figures saved to results/exp23/figures


In [9]:
# Cell 9: Save CSV + JSON
import csv

# --- CSV: one row per passage with all 39 NLL scores + 3 baselines ---
csv_rows = []
for r in all_results:
    bl = r['baselines']
    for p in r['passage_data']:
        row = {
            'query_idx': r['query_idx'],
            'passage_idx': p['passage_idx'],
            'is_relevant': int(p['is_relevant']),
            'word_count': p['word_count'],
            'doc_len': p['doc_len'],
        }
        # 3 baselines
        for t in TARGET_NAMES:
            row[f'baseline_{t}'] = bl[t]
        # 39 NLL scores
        for c in CONDITION_NAMES:
            for t in TARGET_NAMES:
                row[f'{c}_{t}'] = p[f'{c}_{t}']
        csv_rows.append(row)

with open(CSV_PATH, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=csv_rows[0].keys())
    writer.writeheader()
    writer.writerows(csv_rows)
print(f"CSV saved to {CSV_PATH} ({len(csv_rows)} rows)")

# --- JSON: full analysis + per-query results ---
final = {
    'experiment': 'exp23_multi_signal_ranking',
    'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
    'config': {
        'model_name': MODEL_NAME,
        'model_type': 'gemma3',
        'seed': SEED,
        'n_queries': N,
        'total_passages': n_total,
        'n_relevant': n_rel,
        'n_irrelevant': n_irr,
        'layer_cutoff': LAYER_CUTOFF,
        'layer_cutoff_low': LAYER_CUTOFF_LOW,
        'qvi_alphas': QVI_ALPHAS,
        'interp_alpha': INTERP_ALPHA,
        'max_passage_words': MAX_PASSAGE_WORDS,
        'dataset': 'MS MARCO v1.1 validation',
        'conditions': CONDITION_NAMES,
        'targets': TARGET_NAMES,
        'intent_prefixes': INTENT_PREFIXES,
    },
    'analysis': {
        'auc_raw': {k: float(v) for k, v in auc_raw.items()},
        'auc_pmi': {k: float(v) for k, v in auc_pmi.items()},
        'mrr_raw': {k: float(v) for k, v in mrr_raw.items()},
        'mrr_pmi': {k: float(v) for k, v in mrr_pmi.items()},
        'derived_auc': {k: float(v) for k, v in derived_auc.items()},
        'derived_mrr': {k: float(v) for k, v in derived_mrr.items()},
        'two_stage_mrr': {k: float(v) for k, v in two_stage_mrr.items()},
    },
    'per_query_results': all_results,
}

with open(FINAL_RESULTS_PATH, 'w') as f:
    json.dump(final, f, indent=2)
print(f"JSON saved to {FINAL_RESULTS_PATH} ({FINAL_RESULTS_PATH.stat().st_size / 1024:.1f} KB)")

# --- Final Summary ---
print("\n" + "=" * 70)
print("FINAL SUMMARY — Exp 23")
print("=" * 70)
print(f"Model: Gemma 3 4B | 13 conditions × 3 targets = 39 scores/passage")
print(f"Dataset: MS MARCO v1.1 ({N} queries, {n_total} passages)")
print(f"")
print(f"Best AUC (PMI) per target:")
for t in TARGET_NAMES:
    target_aucs = {c: auc_pmi[f'{c}_{t}'] for c in CONDITION_NAMES}
    best = max(target_aucs, key=target_aucs.get)
    bare_auc = auc_pmi[f'bare_{t}']
    print(f"  {t}: bare={bare_auc:.3f} → best={best} ({target_aucs[best]:.3f})")
print(f"")
print(f"Two-stage pipeline MRR@10 (answer target):")
for k in [3, 5]:
    print(f"  k={k}: {two_stage_mrr[f'answer_k{k}']:.3f}")
print(f"  bare PMI: {mrr_pmi['bare_answer']:.3f}")
print("\nDone!")

CSV saved to results/exp23/passage_scores.csv (1692 rows)


JSON saved to results/exp23/results.json (3048.7 KB)

FINAL SUMMARY — Exp 23
Model: Gemma 3 4B | 13 conditions × 3 targets = 39 scores/passage
Dataset: MS MARCO v1.1 (200 queries, 1692 passages)

Best AUC (PMI) per target:
  answer: bare=0.841 → best=oracle_interp (0.842)
  qdoc: bare=0.574 → best=oracle_interp (0.574)
  relevance: bare=0.451 → best=qvi_050 (0.498)

Two-stage pipeline MRR@10 (answer target):
  k=3: 0.866
  k=5: 0.865
  bare PMI: 0.860

Done!
